In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torch.utils.data import Dataset, DataLoader , random_split
from torchvision import utils, transforms
import torch.optim as optim

In [2]:
import import_ipynb
from Deepfake_Dataloader import DeepfakeFaceDataset, ToTensor

In [3]:
INPUT_DIM = 1024
HIDDEN_DIM = 1024
LAYERS = 2

BATCH_SIZE = 1
SEQ_LEN = 1

In [4]:
num_trainset = 1080
csv_path = 'metadata_sample3.csv'
file_path = 'sample/'

transformed_dataset = DeepfakeFaceDataset(csv_file=csv_path,
                                        root_dir=file_path,
                                        transform=transforms.Compose([
                                                ToTensor()
                                           ]))
train_set, val_set = random_split(transformed_dataset, [num_trainset, len(transformed_dataset)-num_trainset])
print(len(transformed_dataset)-num_trainset)
trn_loader = DataLoader(train_set, batch_size=BATCH_SIZE,
                        shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE,
                        shuffle=True)


45


In [5]:
class CNNClassifier(nn.Module):
    
    def __init__(self):
        # 항상 torch.nn.Module을 상속받고 시작
        super(CNNClassifier, self).__init__()
        conv1 = nn.Conv2d(3, 8, (3,3), 1)
        pool1 = nn.MaxPool2d(2)
        conv2 = nn.Conv2d(8, 8, (5,5), 1)
        pool2 = nn.MaxPool2d(2)
        conv3 = nn.Conv2d(8, 16, (5,5), 1)
        pool3 = nn.MaxPool2d(2)
        conv4 = nn.Conv2d(16, 16, (5,5), 1)
        pool4 = nn.MaxPool2d(4)
        
        self.conv_module = nn.Sequential(
            conv1,
            nn.ReLU(),
            pool1,
            conv2,
            nn.ReLU(),
            pool2,
            conv3,
            nn.ReLU(),
            pool3,
            conv4,
            nn.ReLU(),
            pool4
        )
        
        fc1 = nn.Linear(576, 1024)
        dp1 = nn.Dropout(0.5)
        fc2 = nn.Linear(1024, 16)
        d2 = nn.Dropout(0.5)
        fc3 = nn.Linear(16, 1024)

        self.fc_module = nn.Sequential(
            fc1,
            dp1,
            fc2,
            nn.LeakyReLU(),
            d2,
            fc3
        )
        
    def forward(self, x):
        out = self.conv_module(x)
        dim = 1
        for d in out.size()[1:]:
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc_module(out)
        return out

In [6]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, INPUT_DIM, HIDDEN_DIM, N_LAYERS):
        super().__init__()
        self.hidden_dim = HIDDEN_DIM
        self.dropout = nn.Dropout(0.3)
        self.lstm = nn.LSTM(input_size = INPUT_DIM, hidden_size = HIDDEN_DIM, num_layers = N_LAYERS, batch_first=True)
        self.linear = nn.Linear(self.hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.dropout(x)
        #x = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x)
        print(out_pack)
        print(ht.shape)
        out = self.linear(out_pack[-1])
        print(out)
        out = self.sigmoid(out)
        return out

In [7]:
Embedding_Model = CNNClassifier().double()
Embedding_Model.load_state_dict(torch.load('c:/Users/jungw/Desktop/new/GAN/dfdc_pract/deepfake_database/model_2nd.pt'))
print(Embedding_Model)

model = LSTM_variable_input(1024, 1024, 2).double()

model

CNNClassifier(
  (conv_module): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_module): Sequential(
    (0): Linear(in_features=576, out_features=1024, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=1024, out_features=16, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=16, out_features=1024

LSTM_variable_input(
  (dropout): Dropout(p=0.3, inplace=False)
  (lstm): LSTM(1024, 1024, num_layers=2, batch_first=True)
  (linear): Linear(in_features=1024, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [8]:
lr=0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
x = torch.rand([1,3,1024])


In [9]:
x = torch.tensor([[1,2],[3,4]])
z=x.unsqueeze(1)
print(z,z.shape)


tensor([[[1, 2]],

        [[3, 4]]]) torch.Size([2, 1, 2])


In [10]:
num_epochs = 1
num_batches = len(trn_loader)
use_cuda = False

model.train()

trn_loss_list = []
val_loss_list = []
for epoch in range(num_epochs):
    trn_loss = 0.0
    for i,data in enumerate(trn_loader):
        x = torch.DoubleTensor(data['image']/255)
        label = data['label']
        if use_cuda:
            x = torch.cuda.DoubleTensor(x.cuda())
            label = label.cuda()
        
        inputs = Embedding_Model(x.squeeze(0))
        
        model.zero_grad()
        print(inputs.unsqueeze(0).shape)
        output = model(inputs.unsqueeze(0))        
        loss = criterion(output, ((label+1)/2).type(torch.DoubleTensor))
        
        loss.backward()
        optimizer.step()
        
        # trn_loss summary
        trn_loss += loss.item()
        
        # del (memory issue)
        del loss
        del output
        
        if (i+1) % 100 == 0: # every 100 mini-batches
            with torch.no_grad(): # very very very very important!!!
                val_loss = 0.0
                for j, val in enumerate(val_loader):
                    
                    val_x = torch.DoubleTensor(data['image']/255)
                    val_label = data['label']
                    if use_cuda:
                        val_x = torch.cuda.DoubleTensor(val_x.cuda())
                        val_label = val_label.cuda()

                    inputs_val = Embedding_Model(val_x.squeeze(0))
                    
                    # forward propagation
                    output = model(inputs_val.unsqueeze(0))
                    v_loss = criterion(output, ((val_label+1)/2).type(torch.DoubleTensor))
                    
                    val_loss += v_loss
                    
            print("epoch: {}/{} | step: {}/{} | trn loss: {:.8f} | val loss: {:.8f}".format(
                epoch+1, num_epochs, i+1, num_batches, trn_loss / 100, val_loss / len(val_loader)
            ))
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': trn_loss/100
            }, 'c:/Users/jungw/Desktop/new/GAN/dfdc_pract/deepfake_database/rnn_model_1st.pt')
            
            trn_loss_list.append(trn_loss/100)
            val_loss_list.append(val_loss/len(val_loader))
            trn_loss = 0.0

torch.Size([1, 3, 1024])
tensor([[[-0.0262,  0.0846, -0.0062,  ..., -0.0025,  0.1214, -0.0187],
         [ 0.0157, -0.0118,  0.0374,  ...,  0.0055,  0.0373,  0.0013],
         [ 0.0507, -0.0170,  0.0265,  ...,  0.0228,  0.0298, -0.0215]]],
       dtype=torch.float64, grad_fn=<TransposeBackward0>)
torch.Size([2, 1, 1024])
tensor([[-0.0173],
        [-0.0652],
        [-0.0846]], dtype=torch.float64, grad_fn=<AddmmBackward>)


ValueError: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([3, 1])) is deprecated. Please ensure they have the same size.